In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory

# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## importations
import random
import pandas as pd
import gc
import pickle
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance

In [ ]:
## pipeline variables

In [ ]:
## data loading
y2014=pd.read_excel("../input/monoprix/2014.xlsx")
y2015=pd.read_excel("../input/monoprix/2015.xlsx")
y2016=pd.read_excel("../input/monoprix/2016.xlsx")
y2017=pd.read_excel("../input/monoprix/2017.xlsx")
y2018=pd.read_excel("../input/monoprix/2018.xlsx")


In [ ]:
lis=[y2014,y2015,y2016,y2017,y2018]
df=pd.concat(lis,ignore_index=True)

In [ ]:
df.tail()

In [ ]:
l_items=[783480746,611080027,708080011,437000050,646760010,790880881,783480696]

In [ ]:
xx=df[["item_id","id_struct"]].drop_duplicates(["item_id","id_struct"]).set_index(["item_id"]).loc[l_items,:]

In [ ]:
xx

In [ ]:
yy=df[["item_id","id_struct","item_cnt_day"]].sort_values(by='item_cnt_day', ascending=False)

In [ ]:
# data preprocessing
l=[]
old=1
count=0
for i in df.Date:
    if float(i.month)!=float(old):
        old=float(i.month)
        count=count+1
    l.append(count)
df["date_block_num"]=l

In [ ]:
## EDA
df

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
for col in df.columns:
    print(col,df[col].nunique())

In [ ]:
#from pandas_profiling import ProfileReport
#profile = ProfileReport(df.sample(frac=0.01), title="Training Data Profile")


In [ ]:
#profile

In [ ]:
x=df[['Date','date_block_num','shop_id','item_id','item_category','id_struct','Price','item_cnt_day']]
x.set_index(['shop_id', 'item_id'],inplace=True)

In [ ]:
list_of_possibilities=list(set(list(x.index)))
#457

In [ ]:
pair=random.choice(list_of_possibilities)
xx=x.loc[pair]
plt.plot(xx.item_cnt_day.values,xx.Date.values)#'--bo'
plt.ylabel(str(pair))
plt.show()

In [ ]:
xx=x.loc[pair]
plt.plot(xx.Price.values,xx.Date.values)#'--bo'
plt.ylabel(str(pair))
print(pair)
plt.show()

In [ ]:
xx=x.loc[pair]
plt.plot(xx.Price.values,xx.Date.values)#'--bo'
plt.ylabel(str((74, 790880281)))
plt.show()

In [ ]:
#previous value benchmark
X = df[["date_block_num", "shop_id", "item_id", "item_cnt_day"]]

In [ ]:
"""
X["item_cnt_month"] = X.groupby(["date_block_num", "shop_id", "item_id"])["item_cnt_day"].transform(np.sum)
del X["item_cnt_day"]
X.sort_values(by="item_cnt_month",ascending= False)
print(len(X))

X = X.drop_duplicates(["date_block_num", "shop_id", "item_id"])
print(len(X))
"""

In [ ]:
last_month = X["date_block_num"] == 42
last_month_X = X[last_month].copy()
last_month_X = last_month_X.sort_values(by=["shop_id", "item_id"])

last_last_month = X["date_block_num"] == 41
last_last_month_X = X[last_last_month].copy()
last_last_month_X = last_last_month_X.sort_values(by=["shop_id", "item_id"])

In [ ]:
combined = pd.merge(last_month_X,last_last_month_X , on=["shop_id", "item_id"], how="left")
combined = combined.fillna(0)
combined


In [ ]:
bench_data=combined[["item_cnt_day_x","item_cnt_day_y"]]
from sklearn.metrics import mean_squared_error
import math
mse=mean_squared_error(bench_data.item_cnt_day_x,bench_data.item_cnt_day_y)
rmse = math.sqrt(mse)
print(rmse)
#previous value benchmark=9.43


In [ ]:
## data transformation and splitiing and feature engineering
'''
df["item_cnt_month"] = df.groupby(["date_block_num", "shop_id", "item_id"])["item_cnt_day"].transform(np.sum)
df["Price_agg"] = df.groupby(["date_block_num", "shop_id", "item_id"])["Price"].transform(np.mean)
del df["item_cnt_day"]
del df["Price"]
df = df.drop_duplicates(["date_block_num", "shop_id", "item_id"])
'''


In [ ]:
daata=df

In [ ]:
#add empty dates
start=daata.Date.values[0]
end=daata.Date.values[-1]
l=str(start).split('-')
ll=str(end).split('-')
start=l[0]+"-"+l[1]+"-"+l[2][:2]
end=ll[0]+"-"+ll[1]+"-"+ll[2][:2]
d={'Date':pd.date_range(start=start,end=end)}
temp_df = pd.DataFrame(data=d)



In [ ]:
temp=daata[["shop_id","item_id","item_category","id_struct"]].drop_duplicates(["shop_id","item_id"]).reset_index()
#some items are labeled under more than one id_struct

In [ ]:
temp=temp.drop(["index"],axis=1)
temp

In [ ]:
temp1=daata.drop_duplicates(["item_id"],keep='last').reset_index()
temp2=temp1[["item_id","Price"]].set_index(["item_id"])

In [ ]:
temp['key'] = 0
temp_df['key'] = 0

df_cartesian = temp_df.merge(temp, how='outer')

In [ ]:
combination=df_cartesian.drop(["key"],axis=1)

In [ ]:
combination

In [ ]:
temppp=daata.drop(["date_block_num","item_category","id_struct"],axis=1)

In [ ]:
final_comb=pd.merge(combination,temppp , on=["Date","shop_id","item_id"], how="left")

In [ ]:
final_comb

In [ ]:
lis=list(final_comb.Price.fillna(-1).values)
lis1=list(final_comb.item_id.values)

In [ ]:
kk=[lis[x] if lis[x]!=-1 else float(temp2.loc[lis1[x]]) for x in range(len(lis))]


In [ ]:
final=final_comb.drop(["Price"],axis=1)

In [ ]:
final["Price"]=kk
final=final.fillna(0)

In [ ]:
final

In [ ]:
temp_df.reset_index(inplace=True)
temp_df=temp_df.drop(["key"],axis=1)
temp_df

In [ ]:
finals=pd.merge(final,temp_df , on=["Date"], how="left")

In [ ]:
finals

In [ ]:
#generating new features
hol_df=pd.read_csv("../input/tunisian-holidays-2014-2019/holidays.csv")
hol_df["date"] = hol_df["year"].astype(str).str.cat(hol_df[["month", "day_of_month"]].astype(str), sep="-")
hol_df["Date"]=pd.to_datetime(hol_df["date"])
hol_df=hol_df.drop(["date","year","month","day_of_month"],axis=1)
hol_df

In [ ]:
#las=list(final.Date.values)
#las1=list(hol_df.Date.values)
#las01=[1 if x in las1 else 0 for x in las]
hol_df["key"]=1
finals1=pd.merge(finals,hol_df , on=["Date"], how="left")
finals1=finals1.fillna(0)

In [ ]:
finals1["holidays"] = finals1["holiday"].astype('category')
finals1["holidayz"] = finals1["holidays"].cat.codes
finals2=finals1.drop(["holidays","holidayz","holiday"],axis=1)

In [ ]:
#finals2["weekend"]=finals2.Date.dt.dayofweek
#finals2["weekend"]=finals2[["weekend"]].replace({6:1,5:1,4:0,3:0,2:0,1:0})
finals2

In [ ]:
#monthly_Agg
l=[]
old=1
count=0
for i in finals2.Date:
    if float(i.month)!=float(old):
        old=float(i.month)
        count=count+1
    l.append(count)
finals2["date_block_num"]=l
finals2["item_cnt_month"] = finals2.groupby(["date_block_num", "shop_id", "item_id"])["item_cnt_day"].transform(np.sum)
finals2["Price_agg"] = finals2.groupby(["date_block_num", "shop_id", "item_id"])["Price"].transform(np.mean)
finals2["keyz"] = finals2.groupby(["date_block_num", "shop_id", "item_id"])["key"].transform(np.sum)
del finals2["item_cnt_day"]
del finals2["Price"]
del finals2["index"]
del finals2["key"]

finals2 = finals2.drop_duplicates(["date_block_num", "shop_id", "item_id"])
finals2

In [ ]:
finals2.reset_index(inplace=True)
finals2.drop(["index"],axis=1,inplace=True)
finals2

In [ ]:
#cleaning outliers
#plot sales
import matplotlib.pyplot as plt
plt.plot(df.item_cnt_day)
plt.show()
###leeave it for now but talk about it in the meeting

In [ ]:
#feature engineering lag roll expand
grouped_df = finals2.groupby(["shop_id", "item_id"])
df_list=[]
for key,item in grouped_df:
    df_g = grouped_df.get_group(key)
    #lagg features for price and count
    #count
    df_g["item_cnt_month_lag1"]=df_g.item_cnt_month.shift(1).fillna(0)
    df_g["item_cnt_month_lag2"]=df_g.item_cnt_month.shift(2).fillna(0)
    df_g["item_cnt_month_lag3"]=df_g.item_cnt_month.shift(3).fillna(0)
    df_g["item_cnt_month_lag4"]=df_g.item_cnt_month.shift(4).fillna(0)
    df_g["item_cnt_month_lag5"]=df_g.item_cnt_month.shift(5).fillna(0)
    df_g["item_cnt_month_lag6"]=df_g.item_cnt_month.shift(6).fillna(0)
    df_g["item_cnt_month_lag7"]=df_g.item_cnt_month.shift(7).fillna(0)
    #df_g["item_cnt_month_lag8"]=df_g.item_cnt_month.shift(8).fillna(method='bfill')
    #price
    df_g["Price_agg_lag1"]=df_g.Price_agg.shift(1).fillna(0)
    df_g["Price_agg_lag2"]=df_g.Price_agg.shift(2).fillna(0)
    #df_g["Price_agg_lag3"]=df_g.Price_agg.shift(3).fillna(method='bfill')
    #df_g["Price_agg_lag4"]=df_g.Price_agg.shift(4).fillna(method='bfill')
    #rolling window features for count and price
    #count
    #df_g["item_cnt_month_rol1"]=df_g.item_cnt_month.shift(1).rolling(window=2).mean().fillna(method='bfill')
    #price
    #df_g["Price_agg_rol1"]=df_g.Price_agg.shift(1).rolling(window=2).mean().fillna(method='bfill')
    #df_g["Price_agg_rol2"]=df_g.Price_agg.shift(2).rolling(window=3).mean().fillna(method='bfill')
    #df_g["Price_agg_rol3"]=df_g.Price_agg.shift(3).rolling(window=4).mean().fillna(method='bfill')
    #df_g["Price_agg_rol4"]=df_g.Price_agg.shift(4).rolling(window=5).mean().fillna(method='bfill')
    #expanding features for price and count 
    #count
    #df_g["item_cnt_month_exp1"]=df_g.item_cnt_month.shift(1).expanding().mean().fillna(method='bfill')
    #price
    #df_g["Price_agg_exp1"]=df_g.Price_agg.shift(1).expanding().mean().fillna(method='bfill')
    df_list.append(df_g)
new_df=pd.concat(df_list,axis=0)

In [ ]:
new_df.reset_index(inplace=True)
new_df.drop(["index"],axis=1,inplace=True)
new_df

In [ ]:
df0=new_df[["date_block_num","shop_id","item_id","id_struct","item_category","Price_agg","keyz","item_cnt_month_lag1","item_cnt_month_lag2","item_cnt_month_lag3","item_cnt_month_lag4","item_cnt_month_lag5","item_cnt_month_lag6","item_cnt_month_lag7","Price_agg_lag1","Price_agg_lag2"]]
df1=new_df[["item_cnt_month"]]
#"date_block_num","shop_id","item_id","id_struct","item_category","Price_agg","keyz","item_cnt_month_lag1","item_cnt_month_lag2","item_cnt_month_lag3","item_cnt_month_lag4","item_cnt_month_rol1","item_cnt_month_rol2","item_cnt_month_rol3","item_cnt_month_rol4","item_cnt_month_exp1","item_cnt_month_exp2"

In [ ]:
new_df.to_csv("new_base.csv")

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(df0,df1, test_size=0.1,shuffle=False)
#X_val, X_tesst,y_val,y_tesst = train_test_split(X_test,y_test, test_size=0.5,shuffle=False)

In [ ]:
## model
from sklearn.model_selection import cross_val_score
param={'colsample_bytree': 0.8, 'subsample': 0.75, 'eta': 0.02, 'n_estimators': 1100, 'max_depth': 7, 'min_child_weight': 1}
model = XGBRegressor(**param)

model.fit(
    df0, 
    df1, 
    eval_metric="rmse", 
    eval_set=[(df0, df1)], 
    verbose=True, 
    early_stopping_rounds = 1)

pred=model.predict(df0)
mse=mean_squared_error(df1.item_cnt_month,pred)
rmse = math.sqrt(mse)
print("out of fold data results: "+str(rmse))




In [ ]:
l1=list(df1.item_cnt_month.values)
#l1=[x if x<=1500 else 1500 for x in l1]
l2=list(pred)
#l2=[x if x<=1500 else 1500 for x in l2]
l2=[0 if x<0 else round(x) for x in l2]
mse=mean_squared_error(l1,l2)
rmse = math.sqrt(mse)
print(rmse)
import matplotlib.pyplot as plt
plt.plot(l1[-100:])
plt.plot(l2[-100:],'r')
plt.show()

In [ ]:
model

In [ ]:
#plot feature importance
from xgboost import plot_importance
from matplotlib import pyplot
plot_importance(model)
pyplot.show()
'''
#results
base results test rmse: 100,cv score: -148, std cv: 8.81,  features added: keyz
experience 1 results test rmse: 80,cv score: -107, std cv: 9,  features added: keyz, item_cnt_month_lag1
experience 2 results test rmse: 76,cv score: -104.75, std cv: 13,  features added: keyz, item_cnt_month_lag1,item_cnt_month_lag2
experience 3 results test rmse: 67,cv score: -97, std cv: 10,  features added: keyz, item_cnt_month_lag1,item_cnt_month_lag2,item_cnt_month_lag3
experience 4 results test rmse: 66.61,cv score: -97, std cv: 11,  features added: keyz, item_cnt_month_lag1,item_cnt_month_lag2,item_cnt_month_lag3,item_cnt_month_lag4
experience 5 results test rmse: 67,cv score: -97, std cv: 11,  features added: keyz, item_cnt_month_lag1,item_cnt_month_lag2,item_cnt_month_lag3,item_cnt_month_lag4,item_cnt_month_lag5
experience 6 results test rmse: 66.18,cv score: -97.22, std cv: 9.60,  features added: keyz, item_cnt_month_lag1,item_cnt_month_lag2,item_cnt_month_lag3,item_cnt_month_lag4,item_cnt_month_lag5,item_cnt_month_lag6
experience 7 results test rmse: 64.4,cv score: -95.65, std cv: 11.46,  features added: keyz, item_cnt_month_lag1,item_cnt_month_lag2,item_cnt_month_lag3,item_cnt_month_lag4,item_cnt_month_lag5,item_cnt_month_lag6,item_cnt_month_lag7
experience 8 results test rmse: 63.69,cv score: -97.45, std cv: 9.30,  features added: keyz, item_cnt_month_lag1,item_cnt_month_lag2,item_cnt_month_lag3,item_cnt_month_lag4,item_cnt_month_lag5,item_cnt_month_lag6,item_cnt_month_lag7,item_cnt_month_lag8
experience 9 results test rmse: 65.10,cv score: -97, std cv: 10,  features added: keyz, item_cnt_month_lag1,item_cnt_month_lag2,item_cnt_month_lag3,item_cnt_month_lag4,item_cnt_month_lag5,item_cnt_month_lag6,item_cnt_month_lag7,Price_agg_lag1
experience 10 results test rmse: 60.89,cv score:-95.80 , std cv:11 ,  features added: keyz, item_cnt_month_lag1,item_cnt_month_lag2,item_cnt_month_lag3,item_cnt_month_lag4,item_cnt_month_lag5,item_cnt_month_lag6,item_cnt_month_lag7,Price_agg_lag1,Price_agg_lag2
experience 11 results test rmse: 63.23,cv score:-93.64 , std cv:10.84 ,  features added: keyz, item_cnt_month_lag1,item_cnt_month_lag2,item_cnt_month_lag3,item_cnt_month_lag4,item_cnt_month_lag5,item_cnt_month_lag6,item_cnt_month_lag7,Price_agg_lag1,Price_agg_lag2,Price_agg_rol1,Price_agg_rol2,Price_agg_rol3,Price_agg_rol4
experience 12 results test rmse: 63.72,cv score:0 , std cv:0 ,  features added: keyz, item_cnt_month_lag1,item_cnt_month_lag2,item_cnt_month_lag3,item_cnt_month_lag4,item_cnt_month_lag5,item_cnt_month_lag6,item_cnt_month_lag7,Price_agg_lag1,Price_agg_lag2,Price_agg_exp1,Price_agg_exp2


best results : experience 10 + optuna best params + rounding : rmse: 58.99 , cv score : -88.86, std cv: 13.47 , features : keyz, 7 count lags, 2 price lags , 0.1 test validating on train+ 0 filling




'''

In [ ]:
#cross val results
#scores = cross_val_score(model, X_train, y_train,cv=5,scoring="neg_root_mean_squared_error")
#print("Mean cross-validation score: %.2f" % scores.mean())
#print("Std cross-validation score: %.2f" % scores.std())


In [ ]:
#saving the model
import pickle
pickle.dump(model, open("monoprix_forecaster.pickle.dat", "wb"))
model.save_model("xgbmodel")
'''
# some time later...
# load model from file
loaded_model = pickle.load(open("pima.pickle.dat", "rb"))
# make predictions for test data
y_pred = loaded_model.predict(X_test)
'''

In [ ]:
XT=final_comb.drop(["item_cnt_day"],axis=1).drop_duplicates(["shop_id","item_id","item_category"],keep='first').reset_index().drop(["index","Price"],axis=1)


In [ ]:
XT

In [ ]:
#creating the test file for the january 2019
#"date_block_num","shop_id","item_id","id_struct","item_category","Price_agg","keyz","item_cnt_month_lag1","item_cnt_month_lag2","item_cnt_month_lag3","item_cnt_month_lag4","item_cnt_month_lag5","item_cnt_month_lag6","item_cnt_month_lag7","Price_agg_lag1","Price_agg_lag2"
XT["Date"]='2019-01-01'
XT["keyz"]=2
XT["Date"]=pd.to_datetime(XT["Date"])
XT["date_block_num"]=60
XT["Price_agg"]=0
XT["item_cnt_month_lag1"],XT["item_cnt_month_lag2"],XT["item_cnt_month_lag3"],XT["item_cnt_month_lag4"],XT["item_cnt_month_lag5"],XT["item_cnt_month_lag6"],XT["item_cnt_month_lag7"],XT["Price_agg_lag1"],XT["Price_agg_lag2"]=0,0,0,0,0,0,0,0,0

In [ ]:
'''
import datetime
from dateutil import relativedelta
d={'shop_id': '44', 'item_id': '44', 'item_category': '54', 'id_struct': '68', 'price': '4', 'date': '2019-01', 'period': '3'}
period=int(d["period"])
shop_id=d["shop_id"]=int(d["shop_id"])
item_id=d["item_id"]=int(d["item_id"])
item_category=d["item_category"]=int(d["item_category"])
id_struct=d["id_struct"]=int(d["id_struct"])
price=d["price"]=float(d["price"])
date=d["date"]=d["date"]
dd={}
lss=date.split("-")
dates=[datetime.date(int(lss[0]),int(lss[1]), 1)]
for i in range(period-1):
    dates.append(dates[-1] + relativedelta.relativedelta(months=1))
dd["Date"]=dates
dd["shop_id"]=[shop_id]*period
dd["item_id"]=[item_id]*period
dd["item_category"]=[item_category]*period
dd["id_struct"]=[id_struct]*period
dd["Price"]=[price]*period
s = pd.DataFrame(data=dd)
s["Date"]=pd.to_datetime(s["Date"])
dff=hol_df.set_index(['Date'])
dff1=dff.groupby(pd.Grouper(freq="M")).sum()
dff2=dff1.fillna(0).reset_index()
ll1=[x.replace(day=1) for x in dff2.Date]
dff2["Date"]=ll1
dff3=dff2.set_index(['Date'])
kezz=[]
for i in s.Date:
    try:
        kj=int(dff3.loc[i])
        kezz.append(kj)
    except:
        kj=0
        kezz.append(kj)
s["keyz"]=kezz
last_block_num=int(list(new_df.date_block_num.values)[-1])
last_date=list(new_df.Date)[-1]
first_new_date=list(s.Date)[0]
if first_new_date>=last_date:
    num_months = (first_new_date.year - last_date.year) * 12 + (first_new_date.month - last_date.month)
    new_block_num=num_months+last_block_num
else:
    num_months = (last_date.year - first_new_date.year) * 12 + (last_date.month - first_new_date.month)
    new_block_num=last_block_num-num_months
ks=[new_block_num]
for i in range(len(s.Date)-1):
    ks.append(ks[-1]+1)
s["date_block_num"]=ks
s["item_cnt_month_lag1"],s["item_cnt_month_lag2"],s["item_cnt_month_lag3"],s["item_cnt_month_lag4"],s["item_cnt_month_lag5"],s["item_cnt_month_lag6"],s["item_cnt_month_lag7"],s["Price_agg_lag1"],s["Price_agg_lag2"]=0,0,0,0,0,0,0,0,0
ds=new_df[["Date","date_block_num","shop_id","item_id","item_category","id_struct","Price_agg","item_cnt_month"]]
try:
    grouped_XT = ds.groupby(["shop_id", "item_id"]).get_group((shop_id,item_id))
except:
    print("no pair in dataset")
XT_list=[0]*10
for i in range(10):
    try:
        XT_list[i]=list(grouped_XT.item_cnt_month.values)[-(i+1)]
    except:
        break
Xs_list=[0]*5
for i in range(5):
    try:
        Xs_list[i]=list(grouped_XT.Price_agg.values)[-(i+1)]
    except:
        break
for i in range(len(s.Date)):
    s.iloc[i,8:-2]=XT_list[:7]
    XT_list=XT_list[1:]
for i in range(len(s.Date)):
    s.iloc[i,-2:]=Xs_list[:2]
    Xs_list=Xs_list[1:]
    

#extract lags
grouped_XT = XT.groupby(["shop_id", "item_id"])
XT_list=[]
for key,item in grouped_XT:
    XT_g = grouped_XT.get_group(key)
    XD=new_df.groupby(["shop_id", "item_id"]).get_group(key)
    XT_g["Price_agg"]=list(XD.Price_agg.values)[-1]
    XT_g["Price_agg_lag1"]=list(XD.Price_agg.values)[-2]
    XT_g["Price_agg_lag2"]=list(XD.Price_agg.values)[-3]
    XT_g["item_cnt_month_lag1"]=list(XD.item_cnt_month.values)[-1]
    XT_g["item_cnt_month_lag2"]=list(XD.item_cnt_month.values)[-2]
    XT_g["item_cnt_month_lag3"]=list(XD.item_cnt_month.values)[-3]
    XT_g["item_cnt_month_lag4"]=list(XD.item_cnt_month.values)[-4]
    XT_g["item_cnt_month_lag5"]=list(XD.item_cnt_month.values)[-5]
    XT_g["item_cnt_month_lag6"]=list(XD.item_cnt_month.values)[-6]
    XT_g["item_cnt_month_lag7"]=list(XD.item_cnt_month.values)[-7]
    XT_list.append(XT_g)
new_XT=pd.concat(XT_list,axis=0)
X_test=new_XT.reset_index().drop(["Date","index"],axis=1)
loaded_model = pickle.load(open("./monoprix_forecaster.pickle.dat", "rb"))
# make predictions for test data
y_pred = loaded_model.predict(X_test[["date_block_num","shop_id","item_id","id_struct","item_category","Price_agg","keyz","item_cnt_month_lag1","item_cnt_month_lag2","item_cnt_month_lag3","item_cnt_month_lag4","item_cnt_month_lag5","item_cnt_month_lag6","item_cnt_month_lag7","Price_agg_lag1","Price_agg_lag2"]])
import matplotlib.pyplot as plt
plt.plot(y_pred)
plt.show()
X_test["item_cnt_month"]=y_pred
X_test["item_cnt_month"]=round(X_test["item_cnt_month"])
X_test["Date"]='2019-01-01'
X_test["Date"]=pd.to_datetime(X_test["Date"])
X_test[["Date","shop_id","item_id","Price_agg","item_cnt_month"]].to_csv("forecast_2019_01.csv")
'''


In [ ]:
#hyperparams Tuning using Optuna
'''
import optuna
def objective(trial,data=df0,target=df1):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.1,shuffle=False)
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'colsample_bynode': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1]),
        'subsample': trial.suggest_categorical('subsample', [0.75,0.7,0.8,0.9,0.85]),
        'eta': trial.suggest_categorical('eta', [0.01, 0.02,0.2,0.9,0.1,0.11,0.15]),
        'n_estimators': trial.suggest_categorical('n_estimators', [900, 1000,1100,950,1050]),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,10,11,17,20]),
        'seed': 42,
        'min_child_weight': trial.suggest_categorical('min_child_weight', [0.5,1,10,20,100]),
        'booster' : 'gbtree',
        'sampling_method' : "uniform",
    }
    model = XGBRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(train_x,train_y)],eval_metric="rmse",early_stopping_rounds=1,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse
'''

In [ ]:
'''
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
'''

In [ ]:
'''
train1=train[["index","date_block_num","id_struct","item_category","shop_id","item_id","Price","item_cnt_day"]]
train1["shop_id"] = train1["shop_id"].astype('category')
train1["item_id"] = train1["item_id"].astype('category')
train1["index"] = train1["index"].astype('category')
train1["date_block_num"] = train1["date_block_num"].astype('category')
train1["id_struct"] = train1["id_struct"].astype('category')
train1["item_category"] = train1["item_category"].astype('category')
train1["item_cnt_day"] = train1["item_cnt_day"].astype('float64')
train1["Price"] = train1["Price"].astype('float64')
'''

In [ ]:
#tree based
'''
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=100,max_depth=10, random_state=42,verbose=1)
regr.fit(X_train, y_train)
pred=regr.predict(X_test)
mse=mean_squared_error(y_test.item_cnt_day,pred)
rmse = math.sqrt(mse)
print(rmse)
'''
'''
from sklearn.ensemble import AdaBoostRegressor
regr = AdaBoostRegressor(random_state=42, n_estimators=100,learning_rate=0.3, loss='linear')
regr.fit(X_train, y_train)
pred=regr.predict(X_test)
mse=mean_squared_error(y_test.item_cnt_day,pred)
rmse = math.sqrt(mse)
print(rmse)
'''
'''
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=100,
                          learning_rate=1,
                          depth=16)
model.fit(X_train, y_train)
pred=model.predict(X_test)
mse=mean_squared_error(y_test.item_cnt_day,pred)
rmse = math.sqrt(mse)
print(rmse)
'''

In [ ]:
#time series
'''
from fbprophet import Prophet
import numpy as np
prophet_df=df[["Date","shop_id","item_id","item_cnt_month"]]
prophet_df.set_index(['shop_id', 'item_id'],inplace=True)
rez=[]
count=0
for pair in list_of_possibilities:
    print(count)
    try:
        pair_df=prophet_df.loc[pair]
        pair_df.rename(columns={'Date': 'ds', 'item_cnt_day': 'y'},inplace=True)
        ptrain, ptest= train_test_split(pair_df, test_size=0.2,shuffle=False)
        m = Prophet()
        m.fit(ptrain)
        future=ptest[["ds"]]
        forecast = m.predict(future)
        mse=len(ptest.y.values)*mean_squared_error(round(forecast.yhat),ptest.y.values)
        #rmse = math.sqrt(mse)
        rez.append(mse)
        count=count+1
    except ValueError:
        rez.append(0)
        print("THIS PAIR HAS ONLY 2 DATA POINTS OR LESS: "+str(pair))
        count=count+1
print(math.sqrt(np.sum(rez)/len(prophet_df.item_cnt_day.values)))

'''



In [ ]:
'''
arima_df=df[["Date","shop_id","item_id","item_cnt_day"]]
arima_df.set_index(['shop_id', 'item_id'],inplace=True)
pair_df=arima_df.loc[pair]
pair_df["Date"]=pd.to_datetime(pair_df["Date"])
start=pair_df.Date.values[0]
end=pair_df.Date.values[-1]
l=str(start).split('-')
ll=str(end).split('-')
start=l[0]+"-"+l[1]+"-"+l[2][:2]
end=ll[0]+"-"+ll[1]+"-"+ll[2][:2]
d={'Date':pd.date_range(start=start,end=end)}
temp_df = pd.DataFrame(data=d)
comb = pd.merge(temp_df,pair_df , on=["Date"], how="left")
comb = comb.fillna(0)
atrain, atest= train_test_split(pair_df, test_size=0.2,shuffle=False)
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(list(atrain.item_cnt_day.values), order=(5,1,0))
model_fit = model.fit()
output = model_fit.predict(start=len(atrain.item_cnt_day.values)+1,end=len(atrain.item_cnt_day.values)+len(atest.item_cnt_day.values))
mse = len(atest.item_cnt_day.values)*mean_squared_error(atest.item_cnt_day.values, output)
rez.append(mse)
print(math.sqrt(np.sum(rez)/len(arima_df.item_cnt_day.values)))
'''

In [ ]:
#fastai
'''
from fastai.tabular.all import *
to = TabularPandas(train1, procs=[Categorify,Normalize],
                   cat_names = ["index","date_block_num","id_struct","item_category","shop_id","item_id"],
                   cont_names=["Price"],
                   y_names='item_cnt_day')
dls = to.dataloaders(bs=64)
dls.show_batch()
learn = tabular_learner(dls, metrics=rmse)
learn.fit_one_cycle(1)
test_df = test1.copy()
test_df.drop(['item_cnt_day'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)
l=learn.get_preds(dl=dl)
'''

In [ ]:
# data preprocessing
def create_trainable_dataset(df):
    last_block_num=int(list(base.date_block_num.values)[-1])
    last_date=list(base.Date)[-1]
    first_new_date=list(df.Date)[0]
    if first_new_date>=last_date:
        num_months = (first_new_date.year - last_date.year) * 12 + (first_new_date.month - last_date.month)
        new_block_num=num_months+last_block_num
    else:
        num_months = (last_date.year - first_new_date.year) * 12 + (last_date.month - first_new_date.month)
        new_block_num=last_block_num-num_months

    l=[]
    old=1
    #change count
    count=new_block_num
    for i in df.Date:
        if float(i.month)!=float(old):
            old=float(i.month)
            count=count+1
        l.append(count)
    df["date_block_num"]=l

    daata=df

    #add empty dates
    start=daata.Date.values[0]
    end=daata.Date.values[-1]
    l=str(start).split('-')
    ll=str(end).split('-')
    start=l[0]+"-"+l[1]+"-"+l[2][:2]
    end=ll[0]+"-"+ll[1]+"-"+ll[2][:2]
    d={'Date':pd.date_range(start=start,end=end)}
    temp_df = pd.DataFrame(data=d)

    temp=daata[["shop_id","item_id","item_category","id_struct"]].drop_duplicates(["shop_id","item_id"]).reset_index()

    temp=temp.drop(["index"],axis=1)

    temp1=daata.drop_duplicates(["item_id"],keep='last').reset_index()
    temp2=temp1[["item_id","Price"]].set_index(["item_id"])

    temp['key'] = 0
    temp_df['key'] = 0

    df_cartesian = temp_df.merge(temp, how='outer')

    combination=df_cartesian.drop(["key"],axis=1)

    temppp=daata.drop(["date_block_num","item_category","id_struct"],axis=1)

    final_comb=pd.merge(combination,temppp , on=["Date","shop_id","item_id"], how="left")

    lis=list(final_comb.Price.fillna(-1).values)
    lis1=list(final_comb.item_id.values)

    kk=[lis[x] if lis[x]!=-1 else float(temp2.loc[lis1[x]]) for x in range(len(lis))]

    final=final_comb.drop(["Price"],axis=1)

    final["Price"]=kk
    final=final.fillna(0)

    temp_df.reset_index(inplace=True)
    temp_df=temp_df.drop(["key"],axis=1)

    finals=pd.merge(final,temp_df , on=["Date"], how="left")

    #generating new features
    #holy
    holy=pd.read_csv("../input/tunisian-holidays-2014-2019/holidays.csv")
    holy["date"] = holy["year"].astype(str).str.cat(holy[["month", "day_of_month"]].astype(str), sep="-")
    holy["Date"]=pd.to_datetime(holy["date"])
    holy=holy.drop(["date","year","month","day_of_month"],axis=1)
    holy["key"]=1
    finals1=pd.merge(finals,holy , on=["Date"], how="left")
    finals1=finals1.fillna(0)

    finals1["holidays"] = finals1["holiday"].astype('category')
    finals1["holidayz"] = finals1["holidays"].cat.codes
    finals2=finals1.drop(["holidays","holidayz","holiday"],axis=1)

    #monthly_Agg
    l=[]
    old=1
    #change count
    count=new_block_num
    for i in finals2.Date:
        if float(i.month)!=float(old):
            old=float(i.month)
            count=count+1
        l.append(count)
    finals2["date_block_num"]=l
    finals2["item_cnt_month"] = finals2.groupby(["date_block_num", "shop_id", "item_id"])["item_cnt_day"].transform(np.sum)
    finals2["Price_agg"] = finals2.groupby(["date_block_num", "shop_id", "item_id"])["Price"].transform(np.mean)
    finals2["keyz"] = finals2.groupby(["date_block_num", "shop_id", "item_id"])["key"].transform(np.sum)
    del finals2["item_cnt_day"]
    del finals2["Price"]
    del finals2["index"]
    del finals2["key"]

    finals2 = finals2.drop_duplicates(["date_block_num", "shop_id", "item_id"])

    finals2.reset_index(inplace=True)
    finals2.drop(["index"],axis=1,inplace=True)

    #feature engineering lag roll expand
    grouped_df = finals2.groupby(["shop_id", "item_id"])
    df_list=[]
    for key,item in grouped_df:
        df_g = grouped_df.get_group(key)
        #lagg features for price and count
        #count
        df_g["item_cnt_month_lag1"]=df_g.item_cnt_month.shift(1).fillna(0)
        df_g["item_cnt_month_lag2"]=df_g.item_cnt_month.shift(2).fillna(0)
        df_g["item_cnt_month_lag3"]=df_g.item_cnt_month.shift(3).fillna(0)
        df_g["item_cnt_month_lag4"]=df_g.item_cnt_month.shift(4).fillna(0)
        df_g["item_cnt_month_lag5"]=df_g.item_cnt_month.shift(5).fillna(0)
        df_g["item_cnt_month_lag6"]=df_g.item_cnt_month.shift(6).fillna(0)
        df_g["item_cnt_month_lag7"]=df_g.item_cnt_month.shift(7).fillna(0)
        #df_g["item_cnt_month_lag8"]=df_g.item_cnt_month.shift(8).fillna(method='bfill')
        #price
        df_g["Price_agg_lag1"]=df_g.Price_agg.shift(1).fillna(0)
        df_g["Price_agg_lag2"]=df_g.Price_agg.shift(2).fillna(0)
        df_list.append(df_g)
    new_df=pd.concat(df_list,axis=0)

    new_df.reset_index(inplace=True)
    new_df.drop(["index"],axis=1,inplace=True)
    return new_df



In [ ]:
new_df

In [ ]:
df0=new_df[["date_block_num","shop_id","item_id","id_struct","item_category","Price_agg","keyz","item_cnt_month_lag1","item_cnt_month_lag2","item_cnt_month_lag3","item_cnt_month_lag4","item_cnt_month_lag5","item_cnt_month_lag6","item_cnt_month_lag7","Price_agg_lag1","Price_agg_lag2"]]
df1=new_df[["item_cnt_month"]]

from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(df0,df1, test_size=0.1,shuffle=False)

In [ ]:
# some time later...
# load model from file
loaded_model = pickle.load(open("./monoprix_forecaster.pickle.dat", "rb"))


In [ ]:
## model
param={'colsample_bytree': 0.8, 'subsample': 0.75, 'eta': 0.02, 'n_estimators': 1100, 'max_depth': 7, 'min_child_weight': 1}
model = XGBRegressor(**param)

model.fit(
    df0, 
    df1, 
    eval_metric="rmse", 
    eval_set=[(df0, df1)], 
    verbose=True, 
    early_stopping_rounds = 1,xgb_model="./xgbmodel")

pred=model.predict(df0)
mse=mean_squared_error(df1.item_cnt_month,pred)
rmse = math.sqrt(mse)
print("out of fold data results: "+str(rmse))

In [ ]:
model.load_model("./xgbmodel")